<a href="https://colab.research.google.com/github/mannam95/Deep_Learning_Programming/blob/main/IDL_Assignment_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

###Imports

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
tf.__version__

In [ ]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()
print('Training images shape: {}'.format(train_images.shape))
print('Test images shape: {}'.format(test_images.shape))

In [ ]:
for i in range(10):
  plt.subplot(4, 5, i + 1)
  plt.imshow(train_images[i], cmap='gray_r')
  plt.title('Labels : {}'.format(train_labels[i]))
  plt.axis('off')

In [ ]:
class MNISTDataset:
    """'Bare minimum' class to wrap MNIST numpy arrays into a dataset."""
    def __init__(self, train_imgs, train_lbs, test_imgs, test_lbls, batch_size,
                 to01=True, shuffle=True, seed=None):
        """
        Use seed optionally to always get the same shuffling (-> reproducible
        results).
        """
        self.batch_size = batch_size
        self.train_data = train_imgs
        self.train_labels = train_lbs.astype(np.int32)
        self.test_data = test_imgs
        self.test_labels = test_lbls.astype(np.int32)

        if to01:
            # int in [0, 255] -> float in [0, 1]
            self.train_data = self.train_data.astype(np.float32) / 255
            self.test_data = self.test_data.astype(np.float32) / 255

        self.size = self.train_data.shape[0]

        if seed:
            np.random.seed(seed)
        if shuffle:
            self.shuffle_train()
        self.shuffle = shuffle
        self.current_pos = 0

    def next_batch(self):
        """Either gets the next batch, or optionally shuffles and starts a
        new epoch."""
        end_pos = self.current_pos + self.batch_size
        if end_pos < self.size:
            batch = (self.train_data[self.current_pos:end_pos],
                     self.train_labels[self.current_pos:end_pos])
            self.current_pos += self.batch_size
        else:
            # we return what's left (-> possibly smaller batch!) and prepare
            # the start of a new epoch
            batch = (self.train_data[self.current_pos:self.size],
                     self.train_labels[self.current_pos:self.size])
            if self.shuffle:
                self.shuffle_train()
            self.current_pos = 0
            print("Starting new epoch...")
        return batch

    def shuffle_train(self):
        shuffled_inds = np.arange(self.train_data.shape[0])
        np.random.shuffle(shuffled_inds)
        self.train_data = self.train_data[shuffled_inds]
        self.train_labels = self.train_labels[shuffled_inds]

In [ ]:
data = MNISTDataset(train_images.reshape([-1, 784]), train_labels, 
                    test_images.reshape([-1, 784]), test_labels,
                    batch_size=128)

###784(Input)-512(Hidden layer)-10(Output)

In [ ]:
train_steps = 1000
learning_rate = 0.1

#Input layer
W_0 = tf.Variable(np.random.normal(0, 0.1, size=(784,512)).astype(np.float32))
b_0 = tf.Variable(np.random.normal(0, 0.1, size=512).astype(np.float32))

#Hidden layer
W_1 = tf.Variable(np.random.normal(0, 0.1, size=(512,10)).astype(np.float32))
b_1 = tf.Variable(np.random.normal(0, 0.1, size=10).astype(np.float32))


In [ ]:
for step in range(train_steps):
    img_batch, lbl_batch = data.next_batch()
    with tf.GradientTape() as tape:

        h_0 = tf.nn.relu(tf.matmul(img_batch, W_0) + b_0)
        h_1 = tf.nn.relu(tf.matmul(h_0, W_1) + b_1)
        xent = tf.reduce_mean(tf.nn.sparse_softmax_cross_entropy_with_logits(
            logits=h_1, labels=lbl_batch))

    grads = tape.gradient(xent, [W_0,b_0,W_1, b_1])
    W_0.assign_sub(learning_rate * grads[0])
    b_0.assign_sub(learning_rate * grads[1])
    W_1.assign_sub(learning_rate * grads[2])
    b_1.assign_sub(learning_rate * grads[3])
    
    if not step % 100:
        preds = tf.argmax(h_1, axis=1, output_type=tf.int32)
        acc = tf.reduce_mean(tf.cast(tf.equal(preds, lbl_batch),
                             tf.float32))
        print("Loss: {} Accuracy: {}".format(xent, acc))

In [ ]:
test_preds = tf.argmax(tf.matmul(data.test_data, W_0) + b_0, axis=1,
                       output_type=tf.int32)
acc = tf.reduce_mean(tf.cast(tf.equal(test_preds, data.test_labels),
                             tf.float32))
print(acc)